![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/11.2.Pretrained_NER_Profiling_Pipelines.ipynb)

In [ ]:
import json, os
from google.colab import files

license_keys = files.upload()

with open(list(license_keys.keys())[0]) as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)

# Adding license key-value pairs to environment variables
os.environ.update(license_keys)

In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.1.2 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

In [3]:
import json
import os
from pyspark.ml import Pipeline,PipelineModel
from pyspark.sql import SparkSession

from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
import sparknlp_jsl
import sparknlp

params = {"spark.driver.memory":"16G",
"spark.kryoserializer.buffer.max":"2000M",
"spark.driver.maxResultSize":"2000M"}

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

print ("Spark NLP Version :", sparknlp.version())
print ("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark

Spark NLP Version : 3.4.0
Spark NLP_JSL Version : 3.4.0


# Clinical NER Model Profiling Pretrained Pipeline

This pipeline can be used to explore all the available pretrained NER models at once. When you run this pipeline over your text, you will end up with the predictions coming out of each pretrained clinical NER model trained with `embeddings_clinical`.

|Clinical NER Model List|
|-|
|ner_ade_clinical|
|ner_posology_greedy|
|ner_risk_factors|
|jsl_ner_wip_clinical|
|ner_human_phenotype_gene_clinical|
|jsl_ner_wip_greedy_clinical|
|ner_cellular|
|ner_cancer_genetics|
|jsl_ner_wip_modifier_clinical|
|ner_drugs_greedy|
|ner_deid_sd_large|
|ner_diseases|
|nerdl_tumour_demo|
|ner_deid_subentity_augmented|
|ner_jsl_enriched|
|ner_genetic_variants|
|ner_bionlp|
|ner_measurements_clinical|
|ner_diseases_large|
|ner_radiology|
|ner_deid_augmented|
|ner_anatomy|
|ner_chemprot_clinical|
|ner_posology_experimental|
|ner_drugs|
|ner_deid_sd|
|ner_posology_large|
|ner_deid_large|
|ner_posology|
|ner_deidentify_dl|
|ner_deid_enriched|
|ner_bacterial_species|
|ner_drugs_large|
|ner_clinical_large|
|jsl_rd_ner_wip_greedy_clinical|
|ner_medmentions_coarse|
|ner_radiology_wip_clinical|
|ner_clinical|
|ner_chemicals|
|ner_deid_synthetic|
|ner_events_clinical|
|ner_posology_small|
|ner_anatomy_coarse|
|ner_human_phenotype_go_clinical|
|ner_jsl_slim|
|ner_jsl|
|ner_jsl_greedy|
|ner_events_admission_clinical|

You can check [Models Hub](https://nlp.johnsnowlabs.com/models) page for more information about all these models and more.

In [4]:
from sparknlp.pretrained import PretrainedPipeline

clinical_profiling_pipeline = PretrainedPipeline("ner_profiling_clinical", "en", "clinical/models")

ner_profiling_clinical download started this may take some time.
Approx size to download 2.3 GB
[OK!]


In [5]:
text = '''A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM ), one prior episode of HTG-induced pancreatitis three years prior to presentation , associated with an acute hepatitis , and obesity with a body mass index ( BMI ) of 33.5 kg/m2 , presented with a one-week history of polyuria , polydipsia , poor appetite , and vomiting .'''

In [6]:
clinical_result = clinical_profiling_pipeline.annotate(text)
clinical_result.keys()

dict_keys(['ner_ade_clinical_chunks', 'ner_deid_augmented', 'ner_posology_greedy_chunks', 'ner_radiology_wip_clinical', 'ner_deidentify_dl', 'ner_jsl_slim', 'ner_risk_factors_chunks', 'jsl_ner_wip_clinical_chunks', 'ner_deid_synthetic', 'ner_drugs_greedy', 'ner_human_phenotype_gene_clinical_chunks', 'ner_events_admission_clinical', 'jsl_ner_wip_greedy_clinical_chunks', 'ner_posology_greedy', 'ner_cellular_chunks', 'ner_cancer_genetics_chunks', 'ner_jsl_greedy', 'jsl_ner_wip_modifier_clinical_chunks', 'ner_drugs_greedy_chunks', 'ner_deid_sd_large_chunks', 'ner_diseases_chunks', 'ner_diseases_large', 'ner_chemprot_clinical', 'ner_posology_large', 'nerdl_tumour_demo_chunks', 'ner_deid_subentity_augmented_chunks', 'ner_jsl_enriched_chunks', 'ner_genetic_variants_chunks', 'ner_chexpert', 'ner_bionlp_chunks', 'ner_measurements_clinical_chunks', 'ner_diseases_large_chunks', 'ner_drugs_large', 'ner_clinical_large', 'ner_chemicals', 'ner_radiology_chunks', 'ner_bacterial_species', 'ner_deid_aug

In [7]:
print("Clinical Text: \n", text, "\n \nResults:\n")

for i in clinical_result.keys():
  print("{} : ".format(i), clinical_result[i])

Clinical Text: 
 A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM ), one prior episode of HTG-induced pancreatitis three years prior to presentation , associated with an acute hepatitis , and obesity with a body mass index ( BMI ) of 33.5 kg/m2 , presented with a one-week history of polyuria , polydipsia , poor appetite , and vomiting . 
 
Results:

ner_ade_clinical_chunks :  ['pancreatitis', 'acute hepatitis', 'obesity with a body mass index', 'polyuria', 'polydipsia', 'poor appetite', 'vomiting']
ner_deid_augmented :  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 

**Lets show chunk results of NER models in a pandas dataframe.**

In [8]:
import pandas as pd


def get_chunk_results(light_result):

  for i in light_result.keys():
    
    model_name = "_".join(i.split("_")[:-1])

    if (i == 'sentence') | (i == 'token'):
      
      continue
    
    if ("_chunks" not in i):
      
      continue

    elif len(light_result[i]) == 0:
      
      print("\n", "*"*20, model_name, "Model Results", "*"*20)
      print("No Result For This Model")


    else:

      sentences = []
      begins = []
      ends = []
      chunks = []
      entities = []
      confidences = []

      for n in (light_result[i]):

        sentences.append(n.metadata['sentence'])
        begins.append(n.begin)
        ends.append(n.end) 
        chunks.append(n.result)
        entities.append(n.metadata['entity'])
        confidences.append(n.metadata['confidence'])

        
      df = pd.DataFrame({'sentence':sentences, 'begin': begins, 'end': ends, 'chunks': chunks, 'entity': entities, 'confidence': confidences})

      print("\n \n", "*"*20, model_name, "Model Results", "*"*20)
      display(df)

In [9]:
text2 = """
At this time, chest tube placement for drainage of the fluid occurred and thoracoscopy with fluid biopsies, which were performed, which revealed epithelioid malignant mesothelioma. 
The patient was then stained with a PET CT, which showed extensive uptake in the chest, bilateral pleural pericardial effusions, and lymphadenopathy. 
She also had acidic fluid, pectoral and intramammary lymph nodes and uptake in L4 with SUV of 4. This was consistent with stage III disease. Her repeat echocardiogram showed an ejection fraction of 45% to 49%. 
She was transferred to Oncology service and started on chemotherapy on September 1, 2007 with cisplatin 75 mg/centimeter squared equaling 109 mg IV piggyback over 2 hours on September 1, 2007
"""

In [10]:
clinical_full_result = clinical_profiling_pipeline.fullAnnotate(text2)[0]

In [11]:
clinical_full_result['ner_jsl_chunks']

[Annotation(chunk, 15, 34, chest tube placement, {'entity': 'Procedure', 'sentence': '0', 'chunk': '0', 'confidence': '0.47713336'}),
 Annotation(chunk, 40, 60, drainage of the fluid, {'entity': 'Procedure', 'sentence': '0', 'chunk': '1', 'confidence': '0.357225'}),
 Annotation(chunk, 75, 86, thoracoscopy, {'entity': 'Procedure', 'sentence': '0', 'chunk': '2', 'confidence': '0.6137'}),
 Annotation(chunk, 93, 106, fluid biopsies, {'entity': 'Procedure', 'sentence': '0', 'chunk': '3', 'confidence': '0.46585'}),
 Annotation(chunk, 146, 179, epithelioid malignant mesothelioma, {'entity': 'Oncological', 'sentence': '0', 'chunk': '4', 'confidence': '0.75373334'}),
 Annotation(chunk, 219, 224, PET CT, {'entity': 'Test', 'sentence': '1', 'chunk': '5', 'confidence': '0.5252'}),
 Annotation(chunk, 240, 248, extensive, {'entity': 'Modifier', 'sentence': '1', 'chunk': '6', 'confidence': '0.5075'}),
 Annotation(chunk, 250, 255, uptake, {'entity': 'ImagingFindings', 'sentence': '1', 'chunk': '7', 'c

In [12]:
print("Clinical Text:\n", text2)

Clinical Text:
 
At this time, chest tube placement for drainage of the fluid occurred and thoracoscopy with fluid biopsies, which were performed, which revealed epithelioid malignant mesothelioma. 
The patient was then stained with a PET CT, which showed extensive uptake in the chest, bilateral pleural pericardial effusions, and lymphadenopathy. 
She also had acidic fluid, pectoral and intramammary lymph nodes and uptake in L4 with SUV of 4. This was consistent with stage III disease. Her repeat echocardiogram showed an ejection fraction of 45% to 49%. 
She was transferred to Oncology service and started on chemotherapy on September 1, 2007 with cisplatin 75 mg/centimeter squared equaling 109 mg IV piggyback over 2 hours on September 1, 2007



In [13]:
get_chunk_results(clinical_full_result)


 
 ******************** ner_ade_clinical Model Results ********************


,sentence,begin,end,chunks,entity,confidence
0,0,15,34,chest tube placement,DRUG,0.9395667
1,0,40,60,drainage of the fluid,DRUG,0.673325
2,5,600,611,chemotherapy,DRUG,0.9953
3,5,639,647,cisplatin,DRUG,0.9979
4,5,690,701,IV piggyback,DRUG,0.91265



 
 ******************** ner_posology_greedy Model Results ********************


,sentence,begin,end,chunks,entity,confidence
0,5,600,611,chemotherapy,DRUG,0.9973
1,5,639,664,cisplatin 75 mg/centimeter,DRUG,0.6622667
2,5,683,701,109 mg IV piggyback,DRUG,0.70225
3,5,703,714,over 2 hours,DURATION,0.6167



 
 ******************** ner_risk_factors Model Results ********************


...

,sentence,begin,end,chunks,entity,confidence
0,0,15,34,chest tube placement,Procedure,0.47713336
1,0,40,60,drainage of the fluid,Procedure,0.357225
2,0,75,86,thoracoscopy,Procedure,0.6137
3,0,93,106,fluid biopsies,Procedure,0.46585
4,0,146,179,epithelioid malignant mesothelioma,Oncological,0.75373334
5,1,219,224,PET CT,Test,0.5252
6,1,240,248,extensive,Modifier,0.5075
7,1,250,255,uptake,ImagingFindings,0.6767
8,1,264,268,chest,External_body_part_or_region,0.3257
9,1,271,279,bilateral,Direction,0.3233



 
 ******************** ner_jsl_greedy Model Results ********************


,sentence,begin,end,chunks,entity,confidence
0,0,15,34,chest tube placement,Procedure,0.44653335
1,0,75,86,thoracoscopy,Procedure,0.3763
2,0,93,106,fluid biopsies,Procedure,0.536
3,0,146,179,epithelioid malignant mesothelioma,Oncological,0.6796334
4,1,219,224,PET CT,Test,0.4704
5,1,240,268,extensive uptake in the chest,Symptom,0.3656
6,1,271,279,bilateral,Direction,0.2051
7,1,281,309,pleural pericardial effusions,Disease_Syndrome_Disorder,0.2968333
8,1,316,330,lymphadenopathy,Symptom,0.9954
9,2,334,336,She,Gender,0.9987



 
 ******************** ner_events_admission_clinical Model Results ********************


,sentence,begin,end,chunks,entity,confidence
0,0,15,34,chest tube placement,TREATMENT,0.9220667
1,0,40,60,drainage of the fluid,TREATMENT,0.607525
2,0,75,86,thoracoscopy,TEST,0.9681
3,0,93,106,fluid biopsies,TEST,0.9662
4,0,137,144,revealed,EVIDENTIAL,0.9724
5,0,146,179,epithelioid malignant mesothelioma,PROBLEM,0.89350003
6,1,204,210,stained,PROBLEM,0.9988
7,1,217,224,a PET CT,TEST,0.8358
8,1,233,238,showed,EVIDENTIAL,0.5184
9,1,240,268,extensive uptake in the chest,PROBLEM,0.70932


**Now we will show all NER labels of tokens in the same dataframe.**

In [14]:
import pandas as pd

def get_token_results(light_result):

  tokens = [j.result for j in light_result["token"]]
  sentences = [j.metadata["sentence"] for j in light_result["token"]]
  begins = [j.begin for j in light_result["token"]]
  ends = [j.end for j in light_result["token"]]
  model_list = [ a for a in light_result.keys() if (a not in ["sentence", "token"] and "_chunks" not in a)]

  df = pd.DataFrame({'sentence':sentences, 'begin': begins, 'end': ends, 'token':tokens})

  for model_name in model_list:
    
    temp_df = pd.DataFrame(light_result[model_name])
    temp_df["jsl_label"] = temp_df.iloc[:,0].apply(lambda x : x.result)
    temp_df = temp_df[["jsl_label"]]

    # temp_df = get_ner_result(model_name)
    temp_df.columns = [model_name]
    df = pd.concat([df, temp_df], axis=1)
    
  return df

In [15]:
get_token_results(clinical_full_result)

,sentence,begin,end,token,ner_deid_augmented,ner_radiology_wip_clinical,ner_deidentify_dl,ner_jsl_slim,ner_deid_synthetic,ner_drugs_greedy,ner_events_admission_clinical,ner_posology_greedy,ner_jsl_greedy,ner_diseases_large,ner_chemprot_clinical,ner_posology_large,ner_chexpert,ner_drugs_large,ner_clinical_large,ner_chemicals,ner_bacterial_species,ner_bionlp,ner_deid_large,jsl_ner_wip_modifier_clinical,nerdl_tumour_demo,ner_cancer_genetics,ner_posology_small,ner_human_phenotype_gene_clinical,ner_ade_clinical,ner_human_phenotype_go_clinical,ner_risk_factors,ner_clinical,ner_cellular,ner_deid_sd_large,jsl_ner_wip_greedy_clinical,ner_diseases,jsl_rd_ner_wip_greedy_clinical,ner_drugs,ner_anatomy_coarse,ner_medmentions_coarse,ner_deid_sd,ner_anatomy,ner_deid_enriched,ner_deid_subentity_augmented,ner_measurements_clinical,ner_jsl_enriched,ner_posology_experimental,jsl_ner_wip_clinical,ner_jsl,ner_events_clinical,ner_genetic_variants,ner_radiology,ner_posology
0,0,1,2,At,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
1,0,4,7,this,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,B-DATE,O,O,O
2,0,9,12,time,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,I-DATE,O,O,O
3,0,13,13,",",O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
4,0,15,19,chest,O,B-Procedure,O,B-Procedure,O,O,B-TREATMENT,O,B-Procedure,O,O,O,O,O,B-TREATMENT,O,O,B-Tissue,O,B-Procedure,O,O,O,B-HP,B-DRUG,B-GO,O,B-TREATMENT,O,O,B-Procedure,O,B-Procedure,O,B-Anatomy,B-Therapeutic_or_Preventive_Procedure,O,O,O,O,O,B-Procedure,O,B-Procedure,B-Procedure,B-TREATMENT,O,B-Procedure,O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,5,716,717,on,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
124,5,719,727,September,B-DATE,O,B-DATE,B-Date_Time,B-DATE,O,B-DATE,O,B-Date,O,O,O,O,O,O,O,O,O,B-DATE,B-Date,O,O,O,O,O,O,B-PHI,O,O,B-DATE,B-Date,O,B-Date,O,O,O,B-DATE,O,B-DATE,B-DATE,O,B-Date,O,B-Date,B-Date,B-DATE,O,O,O
125,5,729,729,1,I-DATE,O,I-DATE,I-Date_Time,I-DATE,O,I-DATE,O,I-Date,O,O,O,O,O,O,O,O,O,I-DATE,I-Date,O,O,O,O,O,O,O,O,O,I-DATE,I-Date,O,I-Date,O,O,O,I-DATE,O,I-DATE,I-DATE,O,I-Date,O,I-Date,I-Date,I-DATE,O,O,O
126,5,730,730,",",I-DATE,O,I-DATE,I-Date_Time,I-DATE,O,I-DATE,O,I-Date,O,O,O,O,O,O,O,O,O,I-DATE,I-Date,O,O,O,O,O,O,O,O,O,I-DATE,I-Date,O,I-Date,O,O,O,I-DATE,O,I-DATE,I-DATE,O,I-Date,O,I-Date,I-Date,I-DATE,O,O,O


# BioBert NER Model Profiling Pretrained Pipeline

This pipeline can be used to explore all the available pretrained NER models at once. When you run this pipeline over your text, you will end up with the predictions coming out of each pretrained clinical NER model trained with `biobert_pubmed_base_cased`.

|BioBert NER Model List|
|-|
|ner_cellular_biobert|
|ner_diseases_biobert|
|ner_events_biobert|
|ner_bionlp_biobert|
|ner_jsl_greedy_biobert|
|ner_jsl_biobert|
|ner_anatomy_biobert|
|ner_jsl_enriched_biobert|
|ner_human_phenotype_go_biobert|
|ner_deid_biobert|
|ner_deid_enriched_biobert|
|ner_clinical_biobert|
|ner_anatomy_coarse_biobert|
|ner_human_phenotype_gene_biobert|
|ner_posology_large_biobert|
|jsl_rd_ner_wip_greedy_biobert|
|ner_posology_biobert|
|jsl_ner_wip_greedy_biobert|
|ner_chemprot_biobert|
|ner_ade_biobert|
|ner_risk_factors_biobert|

You can check [Models Hub](https://nlp.johnsnowlabs.com/models) page for more information about all these models and more.

In [16]:
from sparknlp.pretrained import PretrainedPipeline

biobert_profiling_pipeline = PretrainedPipeline("ner_profiling_biobert", "en", "clinical/models")

ner_profiling_biobert download started this may take some time.
Approx size to download 715.3 MB
[OK!]


In [17]:
bio_result = biobert_profiling_pipeline.annotate(text)
bio_result.keys()

dict_keys(['ner_ade_biobert', 'ner_cellular_biobert_chunks', 'ner_human_phenotype_go_biobert', 'ner_events_biobert', 'ner_diseases_biobert_chunks', 'ner_events_biobert_chunks', 'jsl_ner_wip_greedy_biobert', 'ner_bionlp_biobert_chunks', 'ner_bionlp_biobert', 'ner_jsl_greedy_biobert_chunks', 'ner_jsl_biobert_chunks', 'ner_jsl_biobert', 'ner_anatomy_biobert_chunks', 'ner_posology_large_biobert', 'ner_jsl_enriched_biobert_chunks', 'jsl_rd_ner_wip_greedy_biobert', 'ner_human_phenotype_go_biobert_chunks', 'ner_posology_biobert', 'ner_deid_biobert_chunks', 'ner_jsl_greedy_biobert', 'ner_chemprot_biobert', 'ner_deid_enriched_biobert_chunks', 'ner_jsl_enriched_biobert', 'token', 'ner_deid_biobert', 'ner_clinical_biobert_chunks', 'ner_human_phenotype_gene_biobert', 'ner_anatomy_coarse_biobert_chunks', 'ner_human_phenotype_gene_biobert_chunks', 'ner_posology_large_biobert_chunks', 'jsl_rd_ner_wip_greedy_biobert_chunks', 'ner_posology_biobert_chunks', 'ner_clinical_biobert', 'ner_diseases_biobert'

In [18]:
print("Clinical Text: \n", text, "\n \nResults:\n")

for i in bio_result.keys():
  print("{} : ".format(i), bio_result[i])

Clinical Text: 
 A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM ), one prior episode of HTG-induced pancreatitis three years prior to presentation , associated with an acute hepatitis , and obesity with a body mass index ( BMI ) of 33.5 kg/m2 , presented with a one-week history of polyuria , polydipsia , poor appetite , and vomiting . 
 
Results:

ner_ade_biobert :  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ADE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ADE', 'I-ADE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ADE', 'O', 'B-ADE', 'O', 'B-ADE', 'I-ADE', 'O', 'O', 'B-ADE', 'O']
ner_cellular_biobert_chunks :  []
ner_human_phenotype_go_biobert :  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',

In [19]:
bio_full_result = biobert_profiling_pipeline.fullAnnotate(text2)[0]

In [20]:
bio_full_result['ner_jsl_biobert_chunks']

[Annotation(chunk, 15, 34, chest tube placement, {'entity': 'Procedure', 'sentence': '0', 'chunk': '0', 'confidence': '0.6034'}),
 Annotation(chunk, 40, 60, drainage of the fluid, {'entity': 'Symptom', 'sentence': '0', 'chunk': '1', 'confidence': '0.37260002'}),
 Annotation(chunk, 75, 86, thoracoscopy, {'entity': 'Procedure', 'sentence': '0', 'chunk': '2', 'confidence': '0.411'}),
 Annotation(chunk, 93, 106, fluid biopsies, {'entity': 'Procedure', 'sentence': '0', 'chunk': '3', 'confidence': '0.57705003'}),
 Annotation(chunk, 146, 179, epithelioid malignant mesothelioma, {'entity': 'Oncological', 'sentence': '0', 'chunk': '4', 'confidence': '0.53636664'}),
 Annotation(chunk, 219, 224, PET CT, {'entity': 'Test', 'sentence': '1', 'chunk': '5', 'confidence': '0.6016'}),
 Annotation(chunk, 240, 255, extensive uptake, {'entity': 'ImagingFindings', 'sentence': '1', 'chunk': '6', 'confidence': '0.33130002'}),
 Annotation(chunk, 264, 268, chest, {'entity': 'External_body_part_or_region', 'sent

In [21]:
print("Clinical Text:\n", text2)

Clinical Text:
 
At this time, chest tube placement for drainage of the fluid occurred and thoracoscopy with fluid biopsies, which were performed, which revealed epithelioid malignant mesothelioma. 
The patient was then stained with a PET CT, which showed extensive uptake in the chest, bilateral pleural pericardial effusions, and lymphadenopathy. 
She also had acidic fluid, pectoral and intramammary lymph nodes and uptake in L4 with SUV of 4. This was consistent with stage III disease. Her repeat echocardiogram showed an ejection fraction of 45% to 49%. 
She was transferred to Oncology service and started on chemotherapy on September 1, 2007 with cisplatin 75 mg/centimeter squared equaling 109 mg IV piggyback over 2 hours on September 1, 2007



**Show NER chunks in a pandas dataframe.**

In [22]:
get_chunk_results(bio_full_result)


 ******************** ner_cellular_biobert Model Results ********************
No Result For This Model

 
 ******************** ner_diseases_biobert Model Results ********************


,sentence,begin,end,chunks,entity,confidence
0,0,146,179,epithelioid malignant mesothelioma,Disease,0.82816666
1,1,281,309,pleural pericardial effusions,Disease,0.8578667
2,1,316,330,lymphadenopathy,Disease,1.0
3,3,456,472,stage III disease,Disease,0.8699



 
 ******************** ner_events_biobert Model Results ********************


,sentence,begin,end,chunks,entity,confidence
0,0,15,34,chest tube placement,TREATMENT,0.86273336
1,0,40,47,drainage,TREATMENT,0.9973
2,0,52,60,the fluid,PROBLEM,0.9656
3,0,75,86,thoracoscopy,TEST,0.5526
4,0,93,106,fluid biopsies,TEST,0.98285
5,0,137,144,revealed,EVIDENTIAL,0.9916
6,0,146,179,epithelioid malignant mesothelioma,PROBLEM,0.9400334
7,1,217,224,a PET CT,TEST,0.8218667
8,1,233,238,showed,EVIDENTIAL,0.7931
9,1,240,268,extensive uptake in the chest,PROBLEM,0.87551993



 
 ******************** ner_bionlp_biobert Model Results ********************


...

,sentence,begin,end,chunks,entity,confidence
0,5,639,647,cisplatin,CHEMICAL,0.78



 
 ******************** ner_ade_biobert Model Results ********************


,sentence,begin,end,chunks,entity,confidence
0,0,15,34,chest tube placement,DRUG,0.9749667
1,0,40,60,drainage of the fluid,DRUG,0.857375
2,0,75,86,thoracoscopy,DRUG,0.9946
3,1,240,268,extensive uptake in the chest,ADE,0.82124007
4,5,600,611,chemotherapy,DRUG,0.5156
5,5,639,647,cisplatin,DRUG,0.8898
6,5,693,701,piggyback,DRUG,1.0



 
 ******************** ner_risk_factors_biobert Model Results ********************


,sentence,begin,end,chunks,entity,confidence
0,5,616,624,September,PHI,0.6927
1,5,719,727,September,PHI,0.7975
2,5,732,735,2007,PHI,0.8128


**Show NER label results in a pandas dataframe.**

In [23]:
get_token_results(clinical_full_result)

,sentence,begin,end,token,ner_deid_augmented,ner_radiology_wip_clinical,ner_deidentify_dl,ner_jsl_slim,ner_deid_synthetic,ner_drugs_greedy,ner_events_admission_clinical,ner_posology_greedy,ner_jsl_greedy,ner_diseases_large,ner_chemprot_clinical,ner_posology_large,ner_chexpert,ner_drugs_large,ner_clinical_large,ner_chemicals,ner_bacterial_species,ner_bionlp,ner_deid_large,jsl_ner_wip_modifier_clinical,nerdl_tumour_demo,ner_cancer_genetics,ner_posology_small,ner_human_phenotype_gene_clinical,ner_ade_clinical,ner_human_phenotype_go_clinical,ner_risk_factors,ner_clinical,ner_cellular,ner_deid_sd_large,jsl_ner_wip_greedy_clinical,ner_diseases,jsl_rd_ner_wip_greedy_clinical,ner_drugs,ner_anatomy_coarse,ner_medmentions_coarse,ner_deid_sd,ner_anatomy,ner_deid_enriched,ner_deid_subentity_augmented,ner_measurements_clinical,ner_jsl_enriched,ner_posology_experimental,jsl_ner_wip_clinical,ner_jsl,ner_events_clinical,ner_genetic_variants,ner_radiology,ner_posology
0,0,1,2,At,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
1,0,4,7,this,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,B-DATE,O,O,O
2,0,9,12,time,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,I-DATE,O,O,O
3,0,13,13,",",O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
4,0,15,19,chest,O,B-Procedure,O,B-Procedure,O,O,B-TREATMENT,O,B-Procedure,O,O,O,O,O,B-TREATMENT,O,O,B-Tissue,O,B-Procedure,O,O,O,B-HP,B-DRUG,B-GO,O,B-TREATMENT,O,O,B-Procedure,O,B-Procedure,O,B-Anatomy,B-Therapeutic_or_Preventive_Procedure,O,O,O,O,O,B-Procedure,O,B-Procedure,B-Procedure,B-TREATMENT,O,B-Procedure,O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,5,716,717,on,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
124,5,719,727,September,B-DATE,O,B-DATE,B-Date_Time,B-DATE,O,B-DATE,O,B-Date,O,O,O,O,O,O,O,O,O,B-DATE,B-Date,O,O,O,O,O,O,B-PHI,O,O,B-DATE,B-Date,O,B-Date,O,O,O,B-DATE,O,B-DATE,B-DATE,O,B-Date,O,B-Date,B-Date,B-DATE,O,O,O
125,5,729,729,1,I-DATE,O,I-DATE,I-Date_Time,I-DATE,O,I-DATE,O,I-Date,O,O,O,O,O,O,O,O,O,I-DATE,I-Date,O,O,O,O,O,O,O,O,O,I-DATE,I-Date,O,I-Date,O,O,O,I-DATE,O,I-DATE,I-DATE,O,I-Date,O,I-Date,I-Date,I-DATE,O,O,O
126,5,730,730,",",I-DATE,O,I-DATE,I-Date_Time,I-DATE,O,I-DATE,O,I-Date,O,O,O,O,O,O,O,O,O,I-DATE,I-Date,O,O,O,O,O,O,O,O,O,I-DATE,I-Date,O,I-Date,O,O,O,I-DATE,O,I-DATE,I-DATE,O,I-Date,O,I-Date,I-Date,I-DATE,O,O,O
